In [ ]:
from pynq import Overlay, MMIO
from time import sleep
from datetime import datetime

class NEC_IR_Controller:
    def __init__(self, bitfile_path,
                 tx0_base=0x41210000,  # AXI GPIO for Transmitter 0
                 tx1_base=0x41220000,  # AXI GPIO for Transmitter 1
                 btn_base=0x41200000   # AXI GPIO for shared Button
                 ):
        # Load overlay
        self.ol = Overlay(bitfile_path)
        self.ol.download()

        # MMIO handles
        self.tx0 = MMIO(tx0_base, 0x10000)
        self.tx1 = MMIO(tx1_base, 0x10000)
        self.btn = MMIO(btn_base, 0x10000)

        # Configure GPIO directions: channel 1 and 2 as outputs
        # AXI GPIO registers: 0x00 (ch1 data), 0x04 (ch1 tri), 0x08 (ch2 data), 0x0C (ch2 tri)
        for ip in (self.tx0, self.tx1, self.btn):
            ip.write(0x04, 0x00000000)  # ch1 as output
            ip.write(0x0C, 0x00000000)  # ch2 as output

        # Timing parameters
        self.NEC_HOLD_TIME = 0.04       # 20 ms hold for NEC frame
        self.BTN_PULSE_WIDTH = 0.001    # 1 ms button pulse

        # Start with clean lines
        self._clear_tx(self.tx0)
        self._clear_tx(self.tx1)
        self._btn_low()

    def _clear_tx(self, tx_ip):
        tx_ip.write(0x00, 0x00000000)  # CH1 low
        tx_ip.write(0x08, 0x00000000)  # CH2 low

    def _btn_high(self):
        self.btn.write(0x00, 0x1)

    def _btn_low(self):
        self.btn.write(0x00, 0x0)

    def send_commands(self, tx0_addr, tx0_cmd, tx1_addr, tx1_cmd):
        """
        Send 8-bit (or up to 32-bit) addr/cmd to TX0 and TX1, then trigger with one button.

        tx*_addr, tx*_cmd: integers (will be written to CH1 and CH2 respectively).
        If you need per-bit parallel buses, pack them into the integer bits appropriately.
        """
        # Clear both transmitters
        self._clear_tx(self.tx0)
        self._clear_tx(self.tx1)
        sleep(self.BTN_PULSE_WIDTH)

        # Write address to CH1 and command to CH2 for each transmitter
        self.tx0.write(0x00, tx0_addr & 0xFF)
        self.tx0.write(0x08, tx0_cmd  & 0xFF)

        self.tx1.write(0x00, tx1_addr & 0xFF)
        self.tx1.write(0x08, tx1_cmd  & 0xFF)

        # Pulse the shared button
        self._btn_high()
        sleep(self.BTN_PULSE_WIDTH)
        self._btn_low()

        # Hold data stable for NEC frame duration
        sleep(self.NEC_HOLD_TIME)

        # Clear lines
        self._clear_tx(self.tx0)
        self._clear_tx(self.tx1)

    def cleanup(self):
        self._clear_tx(self.tx0)
        self._clear_tx(self.tx1)
        self._btn_low()


# Example usage
if __name__ == "__main__":
    ir = NEC_IR_Controller(
        "/home/xilinx/jupyter_notebooks/xilinx/overlays/own/design_1_wrapper.bit",
        tx0_base=0x41210000,
        tx1_base=0x41220000,
        btn_base=0x41200000
    )

    try:
        start = datetime.now()

        # Example 8-bit values (placed in LSBs); adjust width as your IP expects
        tx0_addr = 0x12
        tx0_cmd  = 0x34
        tx1_addr = 0x56
        tx1_cmd  = 0x78

        ir.send_commands(tx0_addr, tx0_cmd, tx1_addr, tx1_cmd)

        print(f"Elapsed: {datetime.now() - start}")

    except Exception as e:
        print(f"Error: {e}")

    finally:
        ir.cleanup()
        print("Cleanup complete")

0:00:25.531279
Cleanup complete
